# Checkpoint Three: Cleaning Data

Now you are ready to clean your data. Before starting coding, provide the link to your dataset below.

My dataset: https://www.kaggle.com/datasets/tonygordonjr/zillow-real-estate-data?utm_source=chatgpt.com&select=listing_mortgage_info.csv

property_listings.csv

listing_subtype.csv

listing_nearby_homes.csv



Import the necessary libraries and create your dataframe(s).

In [5]:
import pandas as pd
import numpy as np

BASE_PATH = "../Data/Final Project Datasets/"

property_df = pd.read_csv(BASE_PATH + "property_listings.csv")
subtype_df = pd.read_csv(BASE_PATH + "listing_subtype.csv")
nearby_df = pd.read_csv(BASE_PATH + "listing_nearby_homes.csv")

property_df.head()


,zpid,price,homeStatus,homeType,datePosted,streetAddress,city,state,zipcode,county,...,rentZestimate,bathrooms,bedrooms,pageViewCount,favoriteCount,propertyTaxRate,timeOnZillow,dateSold,url,lastUpdated
0,32107262.0,750000.0,Recently Sold,Multi Family,2024-03-19,7417 87th Rd,Jamaica,NY,11421.0,Queens County,...,2930.0,2.0,NaN,20.0,0.0,0.86,9 hours,2024-11-24,https://www.zillow.com/homedetails/7417-87th-R...,2024-11-25 09:04:11.007468 UTC
1,20503342.0,3995.0,Recently Sold,Apartment,2024-09-24,1300 Midvale Ave APT 510,Los Angeles,CA,90024.0,Los Angeles County,...,3867.0,2.0,2.0,187.0,5.0,1.16,9 hours,2024-11-24,https://www.zillow.com/homedetails/1300-Midval...,2024-11-25 09:04:11.007468 UTC
2,20183958.0,820000.0,Recently Sold,Single Family,2024-10-27,8300 Capps Ave,Northridge,CA,91324.0,Los Angeles County,...,4540.0,2.0,3.0,21.0,0.0,1.16,9 hours,2024-11-24,https://www.zillow.com/homedetails/8300-Capps-...,2024-11-25 09:04:11.007468 UTC
3,32332472.0,550000.0,Recently Sold,Single Family,2024-07-09,433 Hamden Ave,Staten Island,NY,10306.0,Richmond County,...,2668.0,1.0,2.0,96.0,0.0,0.89,9 hours,2024-11-24,https://www.zillow.com/homedetails/433-Hamden-...,2024-11-25 09:04:11.007468 UTC
4,352427429.0,703478.0,Recently Sold,Single Family,2024-06-19,504 Edwin St #8,Nashville,TN,37207.0,Davidson County,...,3599.0,4.0,4.0,7.0,0.0,0.57,9 hours,2024-11-24,https://www.zillow.com/homedetails/504-Edwin-S...,2024-11-25 09:04:11.007468 UTC


In [6]:
property_df.shape
subtype_df.shape
nearby_df.shape


(239767, 14)

## Missing Data

Test your dataset for missing data and handle it as needed. Make notes in the form of code comments as to your thought process.

In [7]:
# Check for missing values in each dataframe
property_df.isna().sum()
subtype_df.isna().sum()
nearby_df.isna().sum()


zpid                       0
zpidComp                   0
addressComp               18
cityComp                   1
stateComp                  5
zipComp                    8
priceComp                  0
homeTypeComp               0
homeStatusComp             0
livingAreaValueComp    25947
livingAreaUnitsComp    25947
lotAreaValueComp       61529
lotAreaUnitsComp           0
lastUpdated                0
dtype: int64

In [ ]:
# Numeric columns, like: price, beds, baths, sqft, etc. may have missing values.
# Missing numeric values will be filled using the median to avoid skew from outliers.

numeric_cols = property_df.select_dtypes(include=["int64", "float64"]).columns
property_df[numeric_cols] = property_df[numeric_cols].fillna(
    property_df[numeric_cols].median()
)

# For categorical/text columns, missing values likely indicate unavailable data.
# Fill these with 'Unknown' to preserve rows while making missingness explicit.

categorical_cols = property_df.select_dtypes(include=["object"]).columns
property_df[categorical_cols] = property_df[categorical_cols].fillna("Unknown")


In [ ]:
# Missing subtype values are filled with 'Unknown' since they are categorical.

subtype_df = subtype_df.fillna("Unknown")


In [ ]:
# Nearby homes data is supplementary.
# Missing numeric values are filled with 0, assuming absence of nearby listings.

numeric_cols_nearby = nearby_df.select_dtypes(include=["int64", "float64"]).columns
nearby_df[numeric_cols_nearby] = nearby_df[numeric_cols_nearby].fillna(0)

# Missing categorical values are labeled 'Unknown'

categorical_cols_nearby = nearby_df.select_dtypes(include=["object"]).columns
nearby_df[categorical_cols_nearby] = nearby_df[categorical_cols_nearby].fillna("Unknown")


In [11]:
property_df.isna().sum()
subtype_df.isna().sum()
nearby_df.isna().sum()


zpid                   0
zpidComp               0
addressComp            0
cityComp               0
stateComp              0
zipComp                0
priceComp              0
homeTypeComp           0
homeStatusComp         0
livingAreaValueComp    0
livingAreaUnitsComp    0
lotAreaValueComp       0
lotAreaUnitsComp       0
lastUpdated            0
dtype: int64

## Irregular Data

Detect outliers in your dataset and handle them as needed. Use code comments to make notes about your thought process.

In [ ]:
import pandas as pd
import numpy as np


# - I am capping extreme values to reduce distortion while keeping the data.

df = property_df.copy()

# Select numeric columns only.
numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns

outlier_summary = []

for col in numeric_cols:
    # Skip columns that are likely IDs or coordinates where "outliers" are not meaningful
    if any(key in col.lower() for key in ["id", "zip", "zipcode", "lat", "latitude", "lon", "longitude"]):
        continue

    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1

    # If IQR is zero, column may be constant or mostly identical values — skip outlier logic
    if iqr == 0 or pd.isna(iqr):
        continue

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_count = outliers.shape[0]

    outlier_summary.append({
        "column": col,
        "lower_bound": lower_bound,
        "upper_bound": upper_bound,
        "outlier_count": outlier_count
    })

outlier_summary_df = pd.DataFrame(outlier_summary).sort_values("outlier_count", ascending=False)
outlier_summary_df.head(15)


,column,lower_bound,upper_bound,outlier_count
5,bedrooms,1.50,5.50,2035
3,rentZestimate,677.00,4165.00,1834
0,price,-403500.00,1346100.00,1606
7,favoriteCount,-16.00,32.00,1601
6,pageViewCount,-227.50,512.50,1353
4,bathrooms,0.50,4.50,1263
2,livingArea,-287.00,3705.00,982
1,yearBuilt,1864.50,2092.50,46
8,propertyTaxRate,-0.53,2.67,0


In [ ]:
# - Capping reduces the influence of extreme values on averages and models.
# - This keeps rows (listings) in the dataset, which is valuable for analysis.

df_capped = df.copy()

for row in outlier_summary:
    col = row["column"]
    lower = row["lower_bound"]
    upper = row["upper_bound"]

    # Only cap columns where outliers exist
    if row["outlier_count"] > 0:
        df_capped[col] = df_capped[col].clip(lower=lower, upper=upper)

# Replace the original property_df with the cleaned version (or keep both if you prefer)
property_df_clean = df_capped
property_df_clean.head()


,zpid,price,homeStatus,homeType,datePosted,streetAddress,city,state,zipcode,county,...,rentZestimate,bathrooms,bedrooms,pageViewCount,favoriteCount,propertyTaxRate,timeOnZillow,dateSold,url,lastUpdated
0,32107262.0,750000.0,Recently Sold,Multi Family,2024-03-19,7417 87th Rd,Jamaica,NY,11421.0,Queens County,...,2930.0,2.0,3.0,20.0,0.0,0.86,9 hours,2024-11-24,https://www.zillow.com/homedetails/7417-87th-R...,2024-11-25 09:04:11.007468 UTC
1,20503342.0,3995.0,Recently Sold,Apartment,2024-09-24,1300 Midvale Ave APT 510,Los Angeles,CA,90024.0,Los Angeles County,...,3867.0,2.0,2.0,187.0,5.0,1.16,9 hours,2024-11-24,https://www.zillow.com/homedetails/1300-Midval...,2024-11-25 09:04:11.007468 UTC
2,20183958.0,820000.0,Recently Sold,Single Family,2024-10-27,8300 Capps Ave,Northridge,CA,91324.0,Los Angeles County,...,4165.0,2.0,3.0,21.0,0.0,1.16,9 hours,2024-11-24,https://www.zillow.com/homedetails/8300-Capps-...,2024-11-25 09:04:11.007468 UTC
3,32332472.0,550000.0,Recently Sold,Single Family,2024-07-09,433 Hamden Ave,Staten Island,NY,10306.0,Richmond County,...,2668.0,1.0,2.0,96.0,0.0,0.89,9 hours,2024-11-24,https://www.zillow.com/homedetails/433-Hamden-...,2024-11-25 09:04:11.007468 UTC
4,352427429.0,703478.0,Recently Sold,Single Family,2024-06-19,504 Edwin St #8,Nashville,TN,37207.0,Davidson County,...,3599.0,4.0,4.0,7.0,0.0,0.57,9 hours,2024-11-24,https://www.zillow.com/homedetails/504-Edwin-S...,2024-11-25 09:04:11.007468 UTC


In [ ]:
# - Negative values for price, beds, baths, or square footage are not realistic.
# - I will convert them to NaN and then re-fill based on our missing-value strategy.

possible_cols = [c for c in property_df_clean.columns if any(k in c.lower() for k in ["price", "bed", "bath", "sqft", "area"])]

for col in possible_cols:
    if pd.api.types.is_numeric_dtype(property_df_clean[col]):
        property_df_clean.loc[property_df_clean[col] < 0, col] = np.nan

# Re-apply median fill for numeric NaNs created by the cleanup above
numeric_cols_clean = property_df_clean.select_dtypes(include=["int64", "float64"]).columns
property_df_clean[numeric_cols_clean] = property_df_clean[numeric_cols_clean].fillna(
    property_df_clean[numeric_cols_clean].median()
)


In [15]:
# Re-run the outlier summary to confirm outlier counts reduced after capping
df = property_df_clean.copy()
numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns

post_outlier_summary = []

for col in numeric_cols:
    if any(key in col.lower() for key in ["id", "zip", "zipcode", "lat", "latitude", "lon", "longitude"]):
        continue

    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    iqr = q3 - q1

    if iqr == 0 or pd.isna(iqr):
        continue

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    outlier_count = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]

    post_outlier_summary.append({
        "column": col,
        "outlier_count_after": outlier_count
    })

pd.DataFrame(post_outlier_summary).sort_values("outlier_count_after", ascending=False).head(15)


,column,outlier_count_after
0,price,0
1,yearBuilt,0
2,livingArea,0
3,rentZestimate,0
4,bathrooms,0
5,bedrooms,0
6,pageViewCount,0
7,favoriteCount,0
8,propertyTaxRate,0


## Unnecessary Data

Look for the different types of unnecessary data in your dataset and address it as needed. Make sure to use code comments to illustrate your thought process.

In [ ]:
# I will keep what supports analysis and drop what is not useful for our project goals.

df = property_df_clean.copy()


In [17]:
# Check for full duplicate rows
df.duplicated().sum()


np.int64(0)

In [ ]:
# Remove exact duplicate rows 
df = df.drop_duplicates()


In [ ]:
# Checked for duplicate listings by unique identifier (zpid)
# If duplicates exist for the same zpid, keep the most recently updated record.
if "lastUpdated" in df.columns:
    df["lastUpdated"] = pd.to_datetime(df["lastUpdated"], errors="coerce")

# Count duplicated zpids
df["zpid"].duplicated().sum()


np.int64(363)

In [ ]:
# Keep the latest listing per zpid (if there are duplicates)
df = df.sort_values("lastUpdated").drop_duplicates(subset=["zpid"], keep="last")


In [ ]:
# url and streetAddress are very specific and almost every row is unique,
# so they are not helpful for aggregation and trend analysis.
# We'll keep city/state/zipcode/county for geographic analysis instead.

cols_to_drop = ["url", "streetAddress"]
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])


In [ ]:
# livingAreaUnits is often the same value as square footage and doesn't add much insight.
# If it only contains one unique value, I can safely drop it.

if "livingAreaUnits" in df.columns:
    if df["livingAreaUnits"].nunique(dropna=True) <= 1:
        df = df.drop(columns=["livingAreaUnits"])



In [ ]:
# Columns with extremely high missingness may not be reliable for analysis.
# We'll calculate missing % and decide whether to keep them.
missing_percent = (df.isna().sum() / len(df) * 100).sort_values(ascending=False)
missing_percent


dateSold           80.010861
postedYear          0.135759
datePosted          0.135759
postedMonth         0.135759
homeStatus          0.000000
price               0.000000
zpid                0.000000
state               0.000000
city                0.000000
homeType            0.000000
zipcode             0.000000
livingAreaUnits     0.000000
county              0.000000
yearBuilt           0.000000
livingArea          0.000000
bedrooms            0.000000
bathrooms           0.000000
rentZestimate       0.000000
pageViewCount       0.000000
timeOnZillow        0.000000
propertyTaxRate     0.000000
favoriteCount       0.000000
lastUpdated         0.000000
dtype: float64

In [ ]:

# If a column is > 80% missing, it may not be useful unless it's critical to my analysis and it is.

high_missing = missing_percent[missing_percent > 80].index.tolist()
high_missing


['dateSold']

In [26]:
df.shape
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 18415 entries, 16712 to 18
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   zpid             18415 non-null  float64            
 1   price            18415 non-null  float64            
 2   homeStatus       18415 non-null  object             
 3   homeType         18415 non-null  object             
 4   datePosted       18390 non-null  datetime64[ns]     
 5   city             18415 non-null  object             
 6   state            18415 non-null  object             
 7   zipcode          18415 non-null  float64            
 8   county           18415 non-null  object             
 9   yearBuilt        18415 non-null  float64            
 10  livingArea       18415 non-null  float64            
 11  livingAreaUnits  18415 non-null  object             
 12  rentZestimate    18415 non-null  float64            
 13  bathrooms        184

## Inconsistent Data

Check for inconsistent data and address any that arises. As always, use code comments to illustrate your thought process.

In [ ]:
import pandas as pd
import numpy as np


# I standardize formats so grouping, filtering, and merging work reliably.

df = property_df_clean.copy() if "property_df_clean" in globals() else property_df.copy()


In [ ]:
# Columns where I expected consistent categories/labels
text_cols = ["homeStatus", "homeType", "city", "county", "state", "livingAreaUnits"]

for col in text_cols:
    if col in df.columns:
        # Convert to string, strip extra spaces, standardize casing
        # - City/County: Title Case improves readability 
        # - Status/Type: Uppercase makes categories consistent for grouping
        df[col] = df[col].astype(str).str.strip()
        
        if col in ["city", "county"]:
            df[col] = df[col].str.title()
        elif col in ["state"]:
            df[col] = df[col].str.upper()
        else:
            df[col] = df[col].str.upper()


In [ ]:
# Zipcodes may be read as numbers (which can drop leading zeros).

if "zipcode" in df.columns:
    df["zipcode"] = df["zipcode"].astype(str).str.strip()
    # Remove any ".0" if it was imported as a float-like string
    df["zipcode"] = df["zipcode"].str.replace(r"\.0$", "", regex=True)
    # Keep first 5 digits when possible
    df["zipcode"] = df["zipcode"].str.extract(r"(\d{5})", expand=False)


In [ ]:
# Numeric fields can sometimes be stored as strings, especially after cleaning.

numeric_cols = [
    "price", "livingArea", "rentZestimate",
    "bathrooms", "bedrooms",
    "pageViewCount", "favoriteCount",
    "propertyTaxRate", "timeOnZillow", "yearBuilt"
]

for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")


In [ ]:

# Some values are inconsistent because they are impossible or unrealistic.

# Negative values are not valid for these fields
nonnegative_cols = [
    "price", "livingArea", "rentZestimate",
    "bathrooms", "bedrooms",
    "pageViewCount", "favoriteCount", "timeOnZillow"
]
for col in nonnegative_cols:
    if col in df.columns:
        df.loc[df[col] < 0, col] = np.nan

# Bedrooms/bathrooms should be within reasonable limits for typical residential listings
if "bedrooms" in df.columns:
    df.loc[(df["bedrooms"] > 20) | (df["bedrooms"] < 0), "bedrooms"] = np.nan

if "bathrooms" in df.columns:
    df.loc[(df["bathrooms"] > 20) | (df["bathrooms"] < 0), "bathrooms"] = np.nan

# Year built should be within a reasonable historical range
current_year = pd.Timestamp.today().year
if "yearBuilt" in df.columns:
    df.loc[(df["yearBuilt"] < 1800) | (df["yearBuilt"] > current_year), "yearBuilt"] = np.nan


In [ ]:
# Dates may appear in multiple formats; converting to datetime improves analysis.
# Invalid dates will become NaT (missing).

date_cols = ["datePosted", "dateSold", "lastUpdated"]
for col in date_cols:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], errors="coerce")


In [ ]:

# Reviewing unique values helps me spot unexpected labels like "FORSALE" vs "FOR SALE".

for col in ["homeStatus", "homeType", "state"]:
    if col in df.columns:
        display(df[col].value_counts(dropna=False).head(15))


homeStatus
FOR SALE           15003
RECENTLY SOLD       3712
PENDING               30
FOR RENT              13
FORECLOSED             8
PRE FORECLOSURE        8
OTHER                  3
UNKNOWN                1
Name: count, dtype: int64

homeType
SINGLE FAMILY    11304
CONDO             3212
TOWNHOUSE         2300
MULTI FAMILY      1175
LOT                557
MANUFACTURED       158
APARTMENT           71
UNKNOWN              1
Name: count, dtype: int64

state
TX    3288
FL    1820
GA    1606
NY    1561
CA    1499
NV    1214
PA    1162
IL    1043
OH     802
TN     675
NC     569
AZ     566
IN     523
MD     506
KY     424
Name: count, dtype: int64

In [ ]:
# Conversions and invalid-value cleaning may introduce new NaNs.

numeric_cols_in_df = df.select_dtypes(include=["int64", "float64"]).columns
df[numeric_cols_in_df] = df[numeric_cols_in_df].fillna(df[numeric_cols_in_df].median())

df.isna().sum().sort_values(ascending=False)


timeOnZillow       18778
dateSold           15092
zipcode              166
datePosted            28
homeStatus             0
price                  0
zpid                   0
city                   0
streetAddress          0
homeType               0
state                  0
livingArea             0
livingAreaUnits        0
county                 0
yearBuilt              0
bathrooms              0
rentZestimate          0
bedrooms               0
pageViewCount          0
propertyTaxRate        0
favoriteCount          0
url                    0
lastUpdated            0
dtype: int64

## Summarize Your Results

Make note of your answers to the following questions.

1. Did you find all four types of dirty data in your dataset? Yes

Missing data: Several numeric fields such as price, living area, rentZestimate, and yearBuilt contained missing values.

Irregular data (outliers): Extreme values were present in columns like price, living area, page view count, and favorite count.

Unnecessary data: High element count and low value columns such as full street addresses and URLs were identified and removed.

Inconsistent data: Inconsistencies were found in text formatting (case and spacing), zip code formats, and date fields.

2. Did the process of cleaning your data give you new insights into your dataset? Yes

When I cleaned the data it showed me patterns that were not immediately obvious in the raw datasets. Like, the extremely high prices which displayed the luxury and higher end homes instead of the data error. 



3. Is there anything you would like to make note of when it comes to manipulating the data and making visualizations?

Real estate in general has varying and sometimes extreme valaues, which it important to keep outliers as revelent information so it doesnt skew the results. Also, making sure I have consistent data types is importent when it comes to the visualization. 